- state 8,9(in-worst),30(in-best) come in both ratios
- have reverser sorted worstStates
- Doubt is:
    - should I only report statecodes without ant ratios and their ratio21/32 in csv output file
    - what should be the col names

In [ ]:
import pandas as pd

In [ ]:

# for population data
cols=['State', 'Level', 'Name', 'TRU', 'No_HH', 'TOT_P', 'TOT_M', 'TOT_F']
census=pd.read_excel('datasets/census.xlsx',engine='openpyxl',usecols=cols)
censusIndia=census.iloc[0,:]
census=census.loc[census.Level=='STATE']
census=census.loc[census.TRU=='Total']
census=census.append(censusIndia,ignore_index=True)
census.sort_values(by=['State'],axis=0,inplace=True)
census.reset_index(drop=True,inplace=True)

In [ ]:
#===================
# # a function that just takes state code that does
# manipulates and finds appropriate sum
# finds total pop fro that state from census data
# then returns a nice ratio dictionary
#===================

def ratioFinder(stateCode):
    # 5 is specific to TOT_P
    statePop=census.iloc[stateCode,5] 
    # modify state code
    stateCode=str(stateCode).zfill(2)
    # read df from file
    df=pd.read_excel(f'datasets/C-17/{stateCode}.xlsx',skiprows=6,header=None,engine='openpyxl')
    df.fillna(value=0,inplace=True)
    # find total no 
    # note: these col numbers are specific to df #
    total3=sum(df.iloc[:,14].tolist())
    total2=sum(df.iloc[:,9].tolist())-total3 # need to remove total3 as itself is of 2+ language(that also includes 3+ lang persons) speakers
    total1=sum(df.iloc[:,4].tolist())-total2-total3 # as previously I was double substracting 3+ persons I was getting negative number for that
    # doing -toatal3 because total2 has been modified above with toatl3 being substracted
    ratio_1=round(total1/statePop,2)
    ratio_2=round(total2/statePop,2)
    ratio_3=round(total3/statePop,2)
    #store ratios in a dict
    stateRatios={
        'StateCode':stateCode,
        'ratio-1':ratio_1,
        'ratio-2':ratio_2,
        'ratio-3':ratio_3,
    }
    #print('Processed state -- ',stateCode)
    return stateRatios


In [ ]:
%%time
ratioList=[ratioFinder(i) for i in range(0,36)]
ratioDF=pd.DataFrame(ratioList)

In [ ]:
ratioDF21=ratioDF.drop(['ratio-3'],axis=1)
ratioDF21['ratio21']=ratioDF21['ratio-2']/ratioDF21['ratio-1']
ratioDF21.sort_values('ratio21',inplace=True,axis=0,ascending=False)
#print(ratioDF21)
bestStates=ratioDF21.iloc[:3,:]
worstStates=ratioDF21.iloc[-3:,:]
worstStates.sort_values('ratio21',axis=0,ascending=True,inplace=True)
bestWorstDF=pd.concat([bestStates,worstStates],axis=0)
#as que asks for specific ordering
#print(bestWorstDF)
bestWorstDF.to_csv('2-to-1-ratio.csv',index=False)

In [ ]:
ratioDF32=ratioDF.drop(['ratio-1'],axis=1)
ratioDF32['ratio32']=ratioDF32['ratio-3']/ratioDF32['ratio-2']
ratioDF32.sort_values('ratio32',inplace=True,axis=0,ascending=False)
#print(ratioDF32)
bestStates=ratioDF32.iloc[:3,:]
worstStates=ratioDF32.iloc[-3:,:]
worstStates.sort_values('ratio32',axis=0,ascending=True,inplace=True)
bestWorstDF=pd.concat([bestStates,worstStates],axis=0)
#as que asks for specific ordering
#print(bestWorstDF)
bestWorstDF.to_csv('3-to-2-ratio.csv',index=False)